#### SIADS 696 Milestone II

In [184]:
import pandas as pd
import numpy as np
from data_columns import childhood, demographic, diagnosis, all
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
import warnings
warnings.filterwarnings("ignore")


In [157]:
init_df = pd.read_csv('./data/mental-health-comorbidity-raw.csv')
init_df.shape

(5001, 1000)

In [158]:
df = init_df[all()].copy()
# df = df.replace(-9, np.nan)
# df = df.dropna()
df.shape

(5001, 122)

In [159]:
columns2 = demographic() + childhood() + ['DSM_MJD']
num_columns = ["HHSIZE","AGE"] + ["CH13","CH14","CH16","CH21","CH22","CH24","CH38","CH43","CH45"]
child_df = df[columns2].copy()

child_df = child_df.replace(-9, np.nan)
child_df = child_df.dropna()

cat_columns = [col for col in child_df.columns if col not in num_columns]
child_df[cat_columns] = child_df[cat_columns].astype('category')
df[num_columns] = df[num_columns].astype(int)


In [176]:
X = child_df.drop(columns=['DSM_MJD'])
y = child_df['DSM_MJD']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

model = GaussianNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.30      0.34      0.32       197
           5       0.83      0.81      0.82       795

    accuracy                           0.71       992
   macro avg       0.57      0.57      0.57       992
weighted avg       0.72      0.71      0.72       992



In [ ]:
cat_columns = [col for col in cat_columns if col != 'DSM_MJD']

preprocessor = ColumnTransformer([
    ("num", StandardScaler(), num_columns),
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_columns)
])

pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("sampler", RandomOverSampler(random_state=42)),
    # ("classifier", RandomForestClassifier(random_state=42))
    ("classifier", LogisticRegression(solver="saga", max_iter=500))
])

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y, cv=cv, scoring="f1_macro")
print("Cross-val F1 scores:", scores)
print("Mean F1 score:", scores.mean())

Cross-val F1 scores: [0.4887652  0.48465926 0.49950046 0.48463414 0.53273278]
Mean F1 score: 0.49805836775524065
